# import

In [1]:
#export
from fastai.vision import *

In [2]:
#export
import cv2

In [3]:
#export
import os

# functions

In [4]:
#export
def get_y(x, ds_rootdir, imgdir, maskdir):
    yfn = os.path.join(ds_rootdir, maskdir, '%s_mask%s' % (x.stem, x.suffix))
    #print(yfn, x.stem, x.suffix)
    return yfn

In [5]:
#export
class UnetSegmentationLabelList(SegmentationLabelList):
    def open(self, fn):
        return open_mask(fn, div=True)

In [6]:
#export
class UnetSegmentationItemList(SegmentationItemList):
    _label_cls, _square_show_res = UnetSegmentationLabelList, False

In [74]:
#export
def get_databunch(ds_root_dir = 'dataset_20200708', ds_imgdir = 'image'
                  , ds_maskdir = 'mask', bs = 16, valid_pct = 0.2):
    '''
    获取databunch
    参数：
        ds_root_dir：数据集的根目录
        ds_imgdir：图片子目录
        ds_maskdir: mask图片子目录
        bs：batch_size
        valid_pct:验证集百分比
    返回值：
        databunch
    '''
    data = UnetSegmentationItemList.from_folder( \
                    os.path.join(ds_root_dir, ds_imgdir))
    
    data = data.split_by_rand_pct(valid_pct)
    data = data.label_from_func( \
            partial(get_y, ds_rootdir = ds_root_dir, imgdir = ds_imgdir, maskdir = ds_maskdir) \
            , classes=['0', '255'])
    
    #所有的transform除了crop都影响不到ImageSegment.所以flip rotate warp zoom都不能要。
    transforms = get_transforms(do_flip = False
                                , flip_vert = False
                                , max_rotate = 0
                                , max_warp = 0
                                , max_zoom = 1 #不要缩放
                            )
    
    #brightness,contrast对y没啥影响                        
    #transforms = (transforms[0][1:], transforms[1])
    '''
    for tfm in transforms[0]:
        if str(tfm.tfm).find('TfmLighting') >= 0:
            tfm.use_on_y = False
    '''  
    #print(transforms)
    
    #import pdb; pdb.set_trace()        
    data = data.transform(transforms)
    data = data.databunch(bs=bs, num_workers = 0)
    data = data.normalize(imagenet_stats)
    
    '''
    data = (UnetSegmentationItemList
        .from_folder(os.path.join(ds_root_dir, ds_imgdir))
        .split_by_rand_pct(0.2)
        .label_from_func(partial(get_y, ds_rootdir = ds_root_dir, imgdir = ds_imgdir, maskdir = ds_maskdir), classes=['0', '255'])
        .transform(get_transforms(), tfm_y=True)
        .databunch(bs=bs, num_workers = 0)
        .normalize(imagenet_stats)
       )
    '''    
    return data

#data = get_databunch(bs = 4)
#data.show_batch()

# test

In [75]:
data = get_databunch(bs = 4)

In [76]:
data

ImageDataBunch;

Train: LabelList (160 items)
x: UnetSegmentationItemList
Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512)
y: UnetSegmentationLabelList
ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512)
Path: dataset_20200708/image;

Valid: LabelList (40 items)
x: UnetSegmentationItemList
Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512),Image (3, 512, 512)
y: UnetSegmentationLabelList
ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512),ImageSegment (1, 512, 512)
Path: dataset_20200708/image;

Test: None

In [ ]:
data.show_batch(ds_type=DatasetType.Valid)

In [ ]:
data.train_ds[0][1]

# export

In [55]:
!python notebook2script.py --fname 'databunch.ipynb' --outputDir './exp/'

Converted databunch.ipynb to exp/nb_databunch.py
